In [1]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2

In [2]:
model = YOLO("model/best.pt")

In [9]:
response = requests.get("https://media.architecturaldigest.in/wp-content/uploads/2019/05/Design-Experiment-Hyderabad_7-1.jpg")
image = Image.open(BytesIO(response.content))
image = np.asarray(image)

In [10]:
results = model.predict(image)


0: 640x448 1 wc, 128.5ms
Speed: 0.0ms preprocess, 128.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 448)
